## A workshop to develop an options valuation model.

#### The options valuation model should allow me to present historical data regarding an equity and its derivatives (puts and calls) and receive all the information I would need to determine what contract best serves my thesis.

In [1]:
#Pseudocode

#def function_to_determine_tool(the thesis, the toolkit, the bank account):
#The thesis:
#It has a specific defined time period.
#The thesis is instantiated on a specific defined asset. 
#The asset exists on a continuum of possible prices.
#The thesis has an entry and exit strategy.
#The toolkit:
#Is every singular and combinatory tool that can be used on an asset.
#Every single tool has a specific defined time period that must align with the thesis.
#The bank account:
#Helps you understand your capacities.
#Helps you understand your risk tolerance.

#Many of the aspects of all three are correlated and intertwined.

#This is something we must consider as we work.


## <font color = 'pink'> DATA COLLECTION AND EXPLORATION

In [2]:
pip install QuantLib

  Obtaining dependency information for QuantLib from https://files.pythonhosted.org/packages/e0/5a/25ff85724c6736843de961dd715c17e305062d7c5cdfa22d3863239bdeb9/QuantLib-1.37-cp38-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB 1.4 MB/s eta 0:00:10
   - -------------------------------------- 0.5/12.9 MB 8.4 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.9 MB 23.2 MB/s eta 0:00:01
   ------------------ --------------------- 6.0/12.9 MB 35.3 MB/s eta 0:00:01
   ---------------------------- ----------- 9.1/12.9 MB 41.5 MB/s eta 0:00:01
   ------------------------------------ --- 11.8/12.9 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 50.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### <font color = 'pink'> Ultimate Goal: Using just a thesis, a fund size, and an api key, I want the algorithim to select the best strategy of those listed on my CME sheet.

###### <font color = 'pink'> The best strat is determined by median and mean return over monte carlo simulation, best sharpe ratio, and best sterling ratio for now.

In [32]:
#We will use alpha vantage for basic data collection.
#We will focus on AAPL, because it is so liquid and the data is so available.


api_key = 'api_key'

import requests
import pandas as pd
import numpy as np
import QuantLib as ql
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
def get_historical_options_data(ticker, api_key, begin_date, end_date):
    # Initialize an empty dictionary
    date_dict = {}

    # Create a list (or generator) of dates in 'YYYY-MM-DD' format
    all_dates = pd.date_range(begin_date, end_date, freq='D')
    
    # Loop through each date and fetch data
    for single_date in all_dates:
        date_str = single_date.strftime('%Y-%m-%d')
        
        url = f"https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS" \
              f"&symbol={ticker}&date={date_str}&apikey={api_key}"
        r = requests.get(url)
        data = r.json()

        # Some returns may be empty or have errors, so always check
        if 'data' in data:
            df = pd.DataFrame(data['data'])
            # Store this date’s dataframe in the dictionary
            date_dict[date_str] = df
        else:
            # Optionally store an empty DataFrame or log the issue
            date_dict[date_str] = pd.DataFrame()

    master_df = pd.concat(date_dict.values(), ignore_index=True)

    return master_df


In [ ]:
#Just got the idea to use stumpy to see similar historical options flow...

In [57]:
df = get_historical_options_data('AAPL', api_key, '2024-01-01', '2024-01-31')
print(df)

                contractID symbol  expiration  strike  type    last    mark  \
0      AAPL240105C00050000   AAPL  2024-01-05   50.00  call  144.80  135.32   
1      AAPL240105P00050000   AAPL  2024-01-05   50.00   put    0.01    0.01   
2      AAPL240105C00060000   AAPL  2024-01-05   60.00  call    0.00  125.33   
3      AAPL240105P00060000   AAPL  2024-01-05   60.00   put    0.00    0.01   
4      AAPL240105C00065000   AAPL  2024-01-05   65.00  call    0.00  120.33   
...                    ...    ...         ...     ...   ...     ...     ...   
43961  AAPL260618P00290000   AAPL  2026-06-18  290.00   put    0.00  105.25   
43962  AAPL260618C00300000   AAPL  2026-06-18  300.00  call    3.02    3.00   
43963  AAPL260618P00300000   AAPL  2026-06-18  300.00   put  110.00  115.50   
43964  AAPL260618C00310000   AAPL  2026-06-18  310.00  call    2.41    2.50   
43965  AAPL260618P00310000   AAPL  2026-06-18  310.00   put  125.50  125.50   

          bid bid_size     ask ask_size volume open

In [58]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [59]:
#Any option with less volume than 250 over our time period is not worth considering.

def volume_filter(df):
    # 1) Convert volume to numeric
    df['volume'] = pd.to_numeric(df['volume'], errors='coerce').fillna(0)

# 2) Compute sums on the *unfiltered* DataFrame
    df['volume_summed'] = df.groupby('contractID')['volume'].transform('sum')

# 3) Now filter
    df = df[df['volume_summed'] > 250]
    return df

In [60]:
filtered_df = volume_filter(df)
print(filtered_df.head(15))

             contractID symbol  expiration  strike  type   last   mark    bid  \
43  AAPL240105P00160000   AAPL  2024-01-05  160.00   put   0.02   0.01   0.00   
45  AAPL240105P00162500   AAPL  2024-01-05  162.50   put   0.02   0.01   0.00   
47  AAPL240105P00165000   AAPL  2024-01-05  165.00   put   0.02   0.02   0.01   
49  AAPL240105P00167500   AAPL  2024-01-05  167.50   put   0.02   0.03   0.02   
50  AAPL240105C00170000   AAPL  2024-01-05  170.00  call  15.71  15.75  15.20   
51  AAPL240105P00170000   AAPL  2024-01-05  170.00   put   0.03   0.03   0.02   
52  AAPL240105C00172500   AAPL  2024-01-05  172.50  call  12.29  12.95  12.35   
53  AAPL240105P00172500   AAPL  2024-01-05  172.50   put   0.03   0.03   0.01   
54  AAPL240105C00175000   AAPL  2024-01-05  175.00  call  10.80  10.62  10.40   
55  AAPL240105P00175000   AAPL  2024-01-05  175.00   put   0.05   0.06   0.05   
56  AAPL240105C00177500   AAPL  2024-01-05  177.50  call   7.50   8.28   8.00   
57  AAPL240105P00177500   AA

In [68]:
filtered_df.columns

Index(['contractID', 'symbol', 'expiration', 'strike', 'type', 'last', 'mark',
       'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest', 'date',
       'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho',
       'volume_summed'],
      dtype='object')

In [63]:
calls = (filtered_df['type'] == 'call').sum()
puts = (filtered_df['type'] == 'put').sum()

print(f"The number of calls are {calls} and the number of puts are {puts}")

The number of calls are 10202 and the number of puts are 8659


In [67]:
# Sum of puts
sum_of_put_volume = filtered_df[filtered_df['type'] == 'put']['volume'].sum()

# Sum of calls
sum_of_call_volume = filtered_df[filtered_df['type'] == 'call']['volume'].sum()

put_call_ratio = sum_of_put_volume / sum_of_call_volume

print(f"The sum of put volume is {sum_of_put_volume} and the sum of call volume is {sum_of_call_volume}, additionally, the put call ratio is {put_call_ratio}")


The sum of put volume is 7936696 and the sum of call volume is 11234814, additionally, the put call ratio is 0.7064376855727207


In [73]:
print(filtered_df.dtypes)


contractID            object
symbol                object
expiration            object
strike                object
type                  object
last                  object
mark                  object
bid                   object
bid_size              object
ask                   object
ask_size              object
volume                 int64
open_interest         object
date                  object
implied_volatility    object
delta                 object
gamma                 object
theta                 object
vega                  object
rho                   object
volume_summed          int64
dtype: object


In [77]:
filtered_df['strike'] = pd.to_numeric(filtered_df['strike'], errors = 'coerce')
filtered_df['expiration'] = pd.to_datetime(filtered_df['expiration'])
filtered_df['last'] = pd.to_numeric(filtered_df['last'], errors = 'coerce')
filtered_df['bid'] = pd.to_numeric(filtered_df['bid'], errors = 'coerce')
filtered_df['ask'] = pd.to_numeric(filtered_df['ask'], errors = 'coerce')
filtered_df['volume'] = pd.to_numeric(filtered_df['volume'], errors = 'coerce')
filtered_df['open_interest'] = pd.to_numeric(filtered_df['open_interest'], errors = 'coerce')
filtered_df['ask_size'] = pd.to_numeric(filtered_df['ask_size'], errors = 'coerce')
filtered_df['bid_size'] = pd.to_numeric(filtered_df['bid_size'], errors = 'coerce')
filtered_df['mark'] = pd.to_numeric(filtered_df['mark'], errors = 'coerce')
filtered_df['date'] = pd.to_datetime(filtered_df['date'])
filtered_df['implied_volatility'] = pd.to_numeric(filtered_df['implied_volatility'], errors = 'coerce')
filtered_df['delta'] = pd.to_numeric(filtered_df['delta'], errors = 'coerce')
filtered_df['gamma'] = pd.to_numeric(filtered_df['gamma'], errors = 'coerce')
filtered_df['theta'] = pd.to_numeric(filtered_df['theta'], errors = 'coerce')
filtered_df['vega'] = pd.to_numeric(filtered_df['vega'], errors = 'coerce')
filtered_df['rho'] = pd.to_numeric(filtered_df['rho'], errors = 'coerce')



C:\Users\david\AppData\Local\Temp\ipykernel_103556\633977043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['strike'] = pd.to_numeric(filtered_df['strike'], errors = 'coerce')
C:\Users\david\AppData\Local\Temp\ipykernel_103556\633977043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['expiration'] = pd.to_datetime(filtered_df['expiration'])
C:\Users\david\AppData\Local\Temp\ipykernel_103556\633977043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

## Vanna and Charm: Second Order Derivatives

##### I will add these as these second order derivatives can help understand the market and value of the options.

In [78]:
# vanna = vega * (delta/iv)
# charm = (-theta*delta)/Spot price

filtered_df['vanna'] = filtered_df['vega']*(filtered_df['delta']/filtered_df['implied_volatility'])
filtered_df['charm'] = -1*(filtered_df['theta']*filtered_df['delta'])/filtered_df['last']

C:\Users\david\AppData\Local\Temp\ipykernel_103556\2919229364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['vanna'] = filtered_df['vega']*(filtered_df['delta']/filtered_df['implied_volatility'])
C:\Users\david\AppData\Local\Temp\ipykernel_103556\2919229364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['charm'] = -1*(filtered_df['theta']*filtered_df['delta'])/filtered_df['last']


## <font color ='pink'> Now let us create a large script to rank the strategies.

#### First thing first... the thesis.

##### This script is unconcerned with the derivation of a thesis.

##### Still we must understand how the thesis works, and its parts, so that we can model it adequately.

##### So what are the parts of a trade thesis?

##### The exact asset. Apple, Front Month Henry Hub Natural Gas, M2/M1 WTI

##### Intended Direction. For price/vol etc. There are three directions. Up, down, and sideways.

##### Core Rationale. OpenAI API?

##### Defined Time Horizon.

##### Catalysts

##### Fundamental, technical, or quant analysis.

##### Backtesting score

##### Extremely precise entry standards (based on price data, technical or other indicators etc.)

##### Extremely precise exit standards (profit target, time-based exit, invalidation of thesis)

##### Position Sizing and Risk Management: What is the bankroll, what is the VaR, max drawdown constraints.

##### Pain Threshold.

##### Liquidity considerations.

##### Contingency plans, for our purposes, these will be exit strategies.

In [ ]:
#How do we quantify these...

#Exact asset: AAPL
#Asset characteristic: Price
#Intended direction: Up
#Defined Time Horizon: Exactly 30 days.
#Entry Strategy: When price breaks above 30-day MA.
#Exit strategy: Whichever comes first -> 
#     Gain in excess of 7.5%, drawdown in excess of 5%, or 30 days.

#Catalyst: N/A
#AUM: $25,000
#Risk Tolerance: 5% of AUM
#Liquidity Considerations: High, but examined per strategy.


def choose_a_strategy(api_key, ticker, intended_direction, time_horizon, entry_strategy, exit_strategy, catalyst, AUM, risk_tolerance, liquidity):
    
    df = get_historical_options_data(ticker, api_key)
    expiration_dfs = create_expiration_dataframes(df)
    
    
